In [1]:
pip install llama-index-core

  Using cached llama_index_core-0.14.8-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.14.8-py3-none-any.whl (11.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-gemini 0.4.1 requires google-generativeai>=0.5.2, which is not installed.
llama-index-vector-stores-pinecone 0.7.1 requires pinecone<8.0.0,>=7.0.0, but you have pinecone 8.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install llama-index-llms-google-genai


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install llama-index-embeddings-google-genai


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install google-generativeai


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install sentence-transformers


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install llama-index-embeddings-openai

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install llama-index-embeddings-huggingface

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
GEMINI_API_KEY = ""


In [13]:
from llama_index.core import Document, Settings, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from textwrap import shorten
# Use local embeddings (no API costs for this demo)
Settings.embed_model = HuggingFaceEmbedding("sentence-transformers/all-MiniLM-L6-v2")
Settings.llm = None  # No LLM needed yet

LLM is explicitly disabled. Using MockLLM.


In [14]:
# Demo 1 :
# Create documents with rich metadata
docs = [
    Document(
        text="Turmeric coffee (golden latte) combines coffee with turmeric and spices like cinnamon and black pepper. The result is earthy, slightly peppery, and anti-inflammatory.",
        metadata={"heading": "Turmeric Coffee", "category": "Specialty", "region": "India"}
    ),
    Document(
        text="Ashwagandha coffee blends adaptogenic herbs with coffee for sustained energy without jitters. Popular in Ayurvedic wellness circles.",
        metadata={"heading": "Ashwagandha Coffee", "category": "Adaptogen", "region": "India"}
    ),
    Document(
        text="Brewing temperature matters: lighter roasts need 92-96°C while darker roasts can handle 96-100°C. Hotter water extracts more but risks bitterness.",
        metadata={"heading": "Brewing Basics", "category": "Technique", "region": "Universal"}
    ),
]


In [15]:
index = VectorStoreIndex.from_documents(docs)

In [16]:
# Function to peek inside a Node
def inspect_node(index, query, label="Node"):
    """Show the internal structure of a Node"""
    retriever = index.as_retriever(similarity_top_k=1)
    results = retriever.retrieve(query)

    if results:
        node = results[0].node
        print(f"\n{'='*70}")
        print(f"🔍 {label} for query: '{query}'")
        print(f"{'='*70}")
        print(f"📦 Node ID: {node.node_id[:40]}...")
        print(f"📝 Text (truncated):\n   {shorten(node.get_content(), width=200, placeholder='...')}")
        print(f"🏷️  Metadata:")
        for key, val in node.metadata.items():
            print(f"   - {key}: {val}")
        print(f"📊 Relevance Score: {results[0].score:.4f}")


In [12]:
inspect_node(index,"Ashwagandha")


🔍 Node for query: 'Ashwagandha'
📦 Node ID: fdf48f71-3ff3-4479-9f22-3e6d0c4b4bf2...
📝 Text (truncated):
   Ashwagandha coffee blends adaptogenic herbs with coffee for sustained energy without jitters. Popular in Ayurvedic wellness circles.
🏷️  Metadata:
   - heading: Ashwagandha Coffee
   - category: Adaptogen
   - region: India
📊 Relevance Score: 0.6212


In [15]:
# Demo 2 : Four indexes

In [23]:
from llama_index.core import SimpleDirectoryReader, SummaryIndex, KeywordTableIndex, TreeIndex

In [24]:
docs = SimpleDirectoryReader(input_dir="./coffee_pages",required_exts = [".html"]).load_data()

In [25]:
len(docs)

15

In [26]:
# 1.

vector_idx = VectorStoreIndex.from_documents(docs,show_progress=True)

Parsing nodes:   0%|          | 0/15 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/30 [00:00<?, ?it/s]

In [27]:
# 2.
summary_idx = SummaryIndex.from_documents(docs,show_progress=True)

Parsing nodes:   0%|          | 0/15 [00:00<?, ?it/s]

In [28]:
# 3.
keyword_idx = KeywordTableIndex.from_documents(docs,show_progress=True)

Parsing nodes:   0%|          | 0/15 [00:00<?, ?it/s]

Extracting keywords from nodes:   0%|          | 0/30 [00:00<?, ?it/s]

In [29]:
# 4.
tree_idx = TreeIndex.from_documents(docs,show_progress=True)

Parsing nodes:   0%|          | 0/15 [00:00<?, ?it/s]

2025-11-30 16:34:15,262 - INFO - > Building index from nodes: 3 chunks


Generating summaries:   0%|          | 0/3 [00:00<?, ?it/s]

In [30]:
# 4 Index comparisons :
def detailed_comparison(query):
  # 1. Vector
  vec_results = vector_idx.as_retriever(similarity_top_k=2).retrieve(query)
  print("Vector :",vec_results)
  # 2. Summary
  summary_results = summary_idx.as_retriever(similarity_top_k=2).retrieve(query)
  print("Summary :",summary_results)
  # 3. Keyword
  keyword_results = keyword_idx.as_retriever(similarity_top_k=2).retrieve(query)
  print("Keyword :",keyword_results)
  # 4. Tree
  tree_results = tree_idx.as_retriever(similarity_top_k=2).retrieve(query)
  print("Tree :",tree_results)

In [31]:
detailed_comparison("What is Ashwagandha coffee?")

2025-11-30 16:34:40,625 - INFO - > Starting query: What is Ashwagandha coffee?
2025-11-30 16:34:40,629 - INFO - query keywords: ["extract up to 10 keywords from the text. focus on extracting the keywords that we can use to best lookup answers to the question. avoid stopwords.\n---------------------\nwhat is ashwagandha coffee?\n---------------------\nprovide keywords in the following comma-separated format: 'keywords: <keywords>'", 'best', '10', 'separated', 'text', 'question', 'a question is provided below. given the question', 'ashwagandha', 'provided', 'extracting', 'avoid', 'use', 'coffee', 'format', 'keywords', 'answers', 'provide', 'focus', 'comma', 'stopwords', 'following', 'given', 'lookup', 'extract']
2025-11-30 16:34:40,630 - INFO - > Extracted keywords: ['best', '10', 'separated', 'text', 'ashwagandha', 'provided', 'avoid', 'use', 'coffee', 'format', 'keywords', 'provide', 'focus', 'comma', 'stopwords', 'following', 'given', 'extract']
2025-11-30 16:34:40,734 - INFO - >[Leve

Vector : [NodeWithScore(node=TextNode(id_='474f8029-0e2d-47de-b9fb-7fc57fa07529', embedding=None, metadata={'file_path': '/Users/devenderswami/GenAI/GenAI-NoteBooks/coffee_pages/01_ashwagandha_coffee_adaptogenic_latte.html', 'file_name': '01_ashwagandha_coffee_adaptogenic_latte.html', 'file_type': 'text/html', 'file_size': 4757, 'creation_date': '2025-08-17', 'last_modified_date': '2025-08-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='aea63649-ee47-46d6-99d1-e994873f1c26', node_type='4', metadata={'file_path': '/Users/devenderswami/GenAI/GenAI-NoteBooks/coffee_pages/01_ashwagandha_coffee_adaptogenic_latte.html', 'file_name': '01_ashwagandha_coffee_adaptogenic_latte.html', 'file_type': 'text/html', '

In [32]:
def detailed_comparison(query):
    """Deep comparison showing HOW each index type works"""
    print(f"\n{'='*70}")
    print(f"🔍 Query: '{query}'")
    print(f"{'='*70}")

    # 1. VectorStoreIndex (semantic)
    print("\n1️⃣  VectorStoreIndex (Semantic Search via FAISS)")
    print("   How it works: Query → embedding → find closest vectors in FAISS")
    vec_results = vector_idx.as_retriever(similarity_top_k=2).retrieve(query)
    for i, r in enumerate(vec_results, 1):
        print(f"   Match {i} (score: {r.score:.4f}):")
        print(f"   {shorten(r.node.get_content(), width=150, placeholder='...')}")

    # 2. SummaryIndex (overview)
    print("\n2️⃣  SummaryIndex (Pre-computed Summaries)")
    print("   How it works: Return stored summaries (no search needed)")
    sum_results = summary_idx.as_retriever(similarity_top_k=2).retrieve(query)
    for i, r in enumerate(sum_results, 1):
        print(f"   Summary {i}:")
        print(f"   {shorten(r.node.get_content(), width=150, placeholder='...')}")

    # 3. KeywordTableIndex (exact match)
    print("\n3️⃣  KeywordTableIndex (Exact Keyword Match)")
    print("   How it works: Extract keywords from query → lookup in table")
    kw_results = keyword_idx.as_retriever(similarity_top_k=2).retrieve(query)
    for i, r in enumerate(kw_results, 1):
        print(f"   Keyword match {i}:")
        print(f"   {shorten(r.node.get_content(), width=150, placeholder='...')}")

    # 4. TreeIndex (hierarchical)
    print("\n4️⃣  TreeIndex (Hierarchical Navigation)")
    print("   How it works: Traverse tree from root → relevant branch → leaf")
    tree_results = tree_idx.as_retriever(similarity_top_k=2).retrieve(query)
    for i, r in enumerate(tree_results, 1):
        print(f"   Branch {i}:")
        print(f"   {shorten(r.node.get_content(), width=150, placeholder='...')}")

    print(f"\n{'─'*70}")
    print("💡 Notice the differences!")

In [33]:
# Test with different query types
# detailed_comparison("What is Ashwagandha coffee?")
detailed_comparison("What temperature is ideal for light roasts?")


🔍 Query: 'What temperature is ideal for light roasts?'

1️⃣  VectorStoreIndex (Semantic Search via FAISS)
   How it works: Query → embedding → find closest vectors in FAISS
   Match 1 (score: 0.2662):
   It is not medical advice. Individuals with health conditions, pregnant or nursing, or taking medications should consult a qualified professional...
   Match 2 (score: 0.2632):
   It is not medical advice. Individuals with health conditions, pregnant or nursing, or taking medications should consult a qualified professional...

2️⃣  SummaryIndex (Pre-computed Summaries)
   How it works: Return stored summaries (no search needed)
   Summary 1:
   <!DOCTYPE html> <html lang="en"> <head> <meta charset="utf-8" /> <title>Ashwagandha Coffee (Adaptogenic Latte)</title> <meta name="description"...
   Summary 2:
   If using plant milks, choose barista blends for better foam and mouthfeel.</p> </section> <section> <h2>Method</h2> <ol> <li>Warm the milk gently;...
   Summary 3:
   <!DOCTYPE html> 

2025-11-30 16:34:59,214 - INFO - > Starting query: What temperature is ideal for light roasts?
2025-11-30 16:34:59,216 - INFO - query keywords: ['extract', 'light', 'best', '10', 'text', 'a question is provided below. given the question', 'provided', 'question', 'extracting', 'avoid', 'use', "extract up to 10 keywords from the text. focus on extracting the keywords that we can use to best lookup answers to the question. avoid stopwords.\n---------------------\nwhat temperature is ideal for light roasts?\n---------------------\nprovide keywords in the following comma-separated format: 'keywords: <keywords>'", 'format', 'roasts', 'keywords', 'answers', 'provide', 'focus', 'comma', 'stopwords', 'following', 'given', 'temperature', 'ideal', 'lookup', 'separated']
2025-11-30 16:34:59,218 - INFO - > Extracted keywords: ['extract', 'light', 'best', '10', 'text', 'provided', 'avoid', 'use', 'format', 'roasts', 'keywords', 'provide', 'focus', 'comma', 'stopwords', 'following', 'given', 'tempera

   It is not medical advice. Individuals with health conditions, pregnant or nursing, or taking medications should consult a qualified professional...
   Summary 27:
   <!DOCTYPE html> <html lang="en"> <head> <meta charset="utf-8" /> <title>Cold Brew with Indian Spices</title> <meta name="description" content="Cold...
   Summary 28:
   Individuals with health conditions, pregnant or nursing, or taking medications should consult a qualified professional before trying new herbs or...
   Summary 29:
   <!DOCTYPE html> <html lang="en"> <head> <meta charset="utf-8" /> <title>Ayurvedic Lens on Coffee Consumption</title> <meta name="description"...
   Summary 30:
   It is not medical advice. Individuals with health conditions, pregnant or nursing, or taking medications should consult a qualified professional...

3️⃣  KeywordTableIndex (Exact Keyword Match)
   How it works: Extract keywords from query → lookup in table
   Keyword match 1:
   It is not medical advice. Individuals with health co

In [34]:
# Demo 4. Tree Indexes
# Create hierarchical documents (simulating chapters/sections)
hierarchical_docs = [
    Document(
        text="OVERVIEW: This guide covers coffee types (specialty, traditional), brewing methods (temperature, equipment), and health benefits (antioxidants, adaptogens).",
        metadata={"level": "root", "title": "Coffee Guide Overview"}
    ),
    Document(
        text="CHAPTER 1 - COFFEE TYPES: Explores specialty coffees including turmeric lattes, ashwagandha blends, and saffron infusions. Each offers unique flavors and benefits.",
        metadata={"level": "chapter", "title": "Coffee Types", "chapter": 1}
    ),
    Document(
        text="Section 1.1 - Turmeric Coffee: Golden lattes combine turmeric with black pepper for bioavailability. Anti-inflammatory properties make it popular in wellness circles.",
        metadata={"level": "section", "chapter": 1, "section": 1}
    ),
    Document(
        text="Section 1.2 - Ashwagandha Coffee: Adaptogenic herbs reduce stress while coffee provides energy. Popular in Ayurvedic traditions for balanced stimulation.",
        metadata={"level": "section", "chapter": 1, "section": 2}
    ),
    Document(
        text="CHAPTER 2 - BREWING METHODS: Covers optimal temperatures (92-96°C for light roasts, 96-100°C for dark), grind sizes (fine for espresso, coarse for French press), and equipment choices.",
        metadata={"level": "chapter", "title": "Brewing Methods", "chapter": 2}
    ),
    Document(
        text="Section 2.1 - Temperature Control: Light roasts extract best at 92-96°C. Higher temps risk bitterness. Water quality matters as much as temperature.",
        metadata={"level": "section", "chapter": 2, "section": 1}
    ),
]


In [35]:
tree_idx_detailed = TreeIndex.from_documents(hierarchical_docs)

In [36]:
results = tree_idx_detailed.as_retriever(similarity_topk=4).retrieve("What temperature is ideal for light roasts?")

# Try this with all other indexes : Akash to action

2025-11-30 16:35:18,161 - INFO - >[Level 0] Selected node: [1]/[1]


In [37]:
results[0]

NodeWithScore(node=TextNode(id_='d8376eef-87e6-44c8-9cdb-f0c8d40c61c9', embedding=None, metadata={'level': 'root', 'title': 'Coffee Guide Overview'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d085fc9b-87e0-4928-b181-b6800b0e5bb8', node_type='4', metadata={'level': 'root', 'title': 'Coffee Guide Overview'}, hash='e3fd92313c90d6899d8774bf3d8dedc510ccf22eb674ada1a3680024f635a590')}, metadata_template='{key}: {value}', metadata_separator='\n', text='OVERVIEW: This guide covers coffee types (specialty, traditional), brewing methods (temperature, equipment), and health benefits (antioxidants, adaptogens).', mimetype='text/plain', start_char_idx=0, end_char_idx=156, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)

In [38]:
results[0].node.get_content()

'OVERVIEW: This guide covers coffee types (specialty, traditional), brewing methods (temperature, equipment), and health benefits (antioxidants, adaptogens).'

In [39]:
# wihtout tree -> search all the text
# with tree -> navigate smartly

In [40]:
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.llms.google_genai import GoogleGenAI
import numpy as np

In [41]:
# Embeddings :
Settings.embed_model = GoogleGenAIEmbedding(
    model_name="models/text-embedding-004",
    api_key=GEMINI_API_KEY
)

In [42]:
# Embed sample texts
texts = [
    "Turmeric coffee with golden spices",
    "Golden latte with turmeric",  # Similar to above
    "Quantum entanglement in physics",  # Very different
]

print("\n📍 Embedding texts and comparing...")
embeddings = [Settings.embed_model.get_text_embedding(t) for t in texts]

# Calculate cosine similarity
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

sim_1_2 = cosine_similarity(embeddings[0], embeddings[1])
sim_1_3 = cosine_similarity(embeddings[0], embeddings[2])

print(f"\n📊 Similarity Scores:")
print(f"   '{texts[0][:40]}' ↔️")
print(f"   '{texts[1][:40]}'")
print(f"   Similarity: {sim_1_2:.4f} (HIGH - they mean the same!)")

print(f"\n   '{texts[0][:40]}' ↔️")
print(f"   '{texts[2][:40]}'")
print(f"   Similarity: {sim_1_3:.4f} (LOW - totally different!)")

print("\n💡 Key Takeaway: Embeddings capture MEANING, not just words!")

# Set up LLM for remaining demos
Settings.llm = GoogleGenAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0
)
print("\n✅ Gemini LLM configured (gemini-2.5-flash)")

# Rebuild indexes with Gemini embeddings
print("\n🔄 Rebuilding indexes with production Gemini embeddings...")
vector_idx = VectorStoreIndex.from_documents(docs)
print("✅ Indexes rebuilt with Gemini embeddings")




📍 Embedding texts and comparing...


2025-11-30 16:35:37,319 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:35:37,579 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:35:37,901 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"



📊 Similarity Scores:
   'Turmeric coffee with golden spices' ↔️
   'Golden latte with turmeric'
   Similarity: 0.9381 (HIGH - they mean the same!)

   'Turmeric coffee with golden spices' ↔️
   'Quantum entanglement in physics'
   Similarity: 0.5552 (LOW - totally different!)

💡 Key Takeaway: Embeddings capture MEANING, not just words!


2025-11-30 16:35:38,505 - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash "HTTP/1.1 200 OK"



✅ Gemini LLM configured (gemini-2.5-flash)

🔄 Rebuilding indexes with production Gemini embeddings...


2025-11-30 16:35:39,007 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:35:39,393 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:35:39,980 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"


✅ Indexes rebuilt with Gemini embeddings


In [43]:
# normal v/s semnatic chunking :
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser

In [44]:
# normal chunking
normal_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=64)
normal_nodes = normal_splitter.get_nodes_from_documents(docs[:3])

# semantic chunking
semantic_splitter = SemanticSplitterNodeParser(embed_model = Settings.embed_model)
semnatic_nodes = semantic_splitter.get_nodes_from_documents(docs[:3])

2025-11-30 16:35:46,417 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:35:46,742 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:35:47,162 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:35:47,597 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"


In [45]:
from llama_index.core.retrievers import QueryFusionRetriever

In [46]:
import nest_asyncio
nest_asyncio.apply()

# Create base retriever
base_retriever = vector_idx.as_retriever(similarity_top_k=5)

# QueryFusionRetriever automatically generates query variations
fusion_retriever = QueryFusionRetriever(
    retrievers=[base_retriever],
    similarity_top_k=5,
    num_queries=3,  # Generate 3 variations
    mode="reciprocal_rerank",
    use_async=False,
    llm=Settings.llm
)

print("✅ Multi-query retriever created")
print("   - Generates 3 query variations automatically")
print("   - Retrieves with each variation")
print("   - Fuses results with reciprocal ranking")

# Test it
query = "What are the benefits of Ashwagandha?"
print(f"\n🔍 Original Query: '{query}'")

# Compare single vs multi-query
single_results = base_retriever.retrieve(query)
multi_results = fusion_retriever.retrieve(query)

print(f"\n📊 Results Comparison:")
print(f"   Single query: {len(single_results)} documents")
print(f"   Multi-query:  {len(multi_results)} unique documents")
print(f"   Improvement:  +{len(multi_results) - len(single_results)} docs")

✅ Multi-query retriever created
   - Generates 3 query variations automatically
   - Retrieves with each variation
   - Fuses results with reciprocal ranking

🔍 Original Query: 'What are the benefits of Ashwagandha?'


2025-11-30 16:35:52,299 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:35:52,316 - INFO - AFC is enabled with max remote calls: 10.
2025-11-30 16:35:54,681 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-11-30 16:35:54,923 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:35:55,268 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:35:55,524 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"



📊 Results Comparison:
   Single query: 5 documents
   Multi-query:  5 unique documents
   Improvement:  +0 docs


In [49]:
query = "What are the benefits of Ashwagandha?"
queries = fusion_retriever._get_queries(query)

print("\n🧩 Query Variations Generated:")
for i, q in enumerate(queries, 1):
    print(f"{i}. {q}")


2025-11-30 16:37:13,721 - INFO - AFC is enabled with max remote calls: 10.
2025-11-30 16:37:16,993 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"



🧩 Query Variations Generated:
1. Ashwagandha health benefits
2. Uses and effects of Ashwagandha


In [50]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


# -----------------------------
# 1. Create Query Engines
# -----------------------------
vector_qe = vector_idx.as_query_engine(similarity_top_k=4)
keyword_qe = keyword_idx.as_query_engine()
summary_qe = summary_idx.as_query_engine()


# -----------------------------
# 2. Wrap them as Tools
# -----------------------------
tools = [
    QueryEngineTool.from_defaults(
        query_engine=vector_qe,
        description="Use for general questions about coffee types and ingredients.",
        name="vector_engine"
    ),
    QueryEngineTool.from_defaults(
        query_engine=keyword_qe,
        description="Use for finding exact keywords or specific terms.",
        name="keyword_engine"
    ),
    QueryEngineTool.from_defaults(
        query_engine=summary_qe,
        description="Use for high-level overviews and summaries.",
        name="summary_engine"
    )
]


# -----------------------------
# 3. Create Router
# -----------------------------
selector = LLMSingleSelector.from_defaults(llm=Settings.llm)

router = RouterQueryEngine.from_defaults(
    query_engine_tools=tools,
    selector=selector,
    llm=Settings.llm,
    verbose = True
)

print("✅ Router created with 3 specialized engines")


# -----------------------------
# 4. Test Routing (Clean)
# -----------------------------
test_queries = [
    "What is turmeric coffee?", # vector search
    "Find entries with ashwagandha", # keyword search
    "Give me an overview of these documents" # summary search
]

print("\n🔀 Testing Query Routing...")

for q in test_queries:
    ans = router.query(q)
    print(f"\nQuery: {q}")
    print("Answer:", str(ans)[:200], "...")


2025-11-30 16:37:51,733 - INFO - AFC is enabled with max remote calls: 10.


✅ Router created with 3 specialized engines

🔀 Testing Query Routing...


2025-11-30 16:37:56,531 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-11-30 16:37:56,542 - INFO - Selecting query engine 0: The question 'What is turmeric coffee?' asks for a description or definition of a specific type of coffee. Choice (1) explicitly states 'Use for general questions about coffee types and ingredients,' which directly aligns with inquiring about 'turmeric coffee' as a 'coffee type' and implicitly about its 'ingredients' (turmeric). Choices (2) and (3) are less relevant as the question is not about finding exact keywords nor is it exclusively seeking a high-level overview without details about its nature or components..


Selecting query engine 0: The question 'What is turmeric coffee?' asks for a description or definition of a specific type of coffee. Choice (1) explicitly states 'Use for general questions about coffee types and ingredients,' which directly aligns with inquiring about 'turmeric coffee' as a 'coffee type' and implicitly about its 'ingredients' (turmeric). Choices (2) and (3) are less relevant as the question is not about finding exact keywords nor is it exclusively seeking a high-level overview without details about its nature or components..


2025-11-30 16:37:56,867 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:37:56,896 - INFO - AFC is enabled with max remote calls: 10.
2025-11-30 16:37:58,722 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-11-30 16:37:58,733 - INFO - AFC is enabled with max remote calls: 10.


...


Query: Find entries with ashwagandha
Answer: The following entries include information about ashwagandha:

*   Ashwagandha Coffee (Adaptogenic Latte)
*   Mushroom & Ashwagandha Coffee (Adaptogen Blend) ...


2025-11-30 16:38:10,435 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-11-30 16:38:10,443 - INFO - Selecting query engine 2: The question 'Give me an overview of these documents' directly asks for an overview. Choice (3) explicitly states 'Use for high-level overviews and summaries', which perfectly matches the intent of the question..
2025-11-30 16:38:10,500 - INFO - AFC is enabled with max remote calls: 10.


Selecting query engine 2: The question 'Give me an overview of these documents' directly asks for an overview. Choice (3) explicitly states 'Use for high-level overviews and summaries', which perfectly matches the intent of the question..


2025-11-30 16:38:14,473 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"



Query: Give me an overview of these documents
Answer: The documents provide a collection of information on various Indian coffee preparations and related topics. They cover recipes and cultural notes for specific coffee types such as Ashwagandha Coffee ( ...


In [51]:
# =============================================================================
# DEMO 9: Post-Processing Pipeline
# =============================================================================
print("\n" + "="*70)
print("DEMO 9: Post-Processing Pipeline (Rerank + Dedup)")
print("="*70)

from llama_index.core.postprocessor import SimilarityPostprocessor, SentenceTransformerRerank

# Create post-processors
print("🔧 Setting up post-processors...")

reranker = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-6-v2",
    top_n=5
)
print("   ✅ Reranker (cross-encoder)")

deduplicator = SimilarityPostprocessor(similarity_cutoff=0.85)
print("   ✅ Deduplicator (similarity threshold 0.85)")

# Build query engine with post-processing
advanced_qe = vector_idx.as_query_engine(
    similarity_top_k=20,  # Retrieve more initially
    node_postprocessors=[
        reranker,      # Rerank to top 3
        deduplicator   # Remove duplicates
    ],
    response_mode="compact"
)


print("   1. Retrieve top 10 candidates")
print("   2. Rerank with cross-encoder to top 3")
print("   3. Deduplicate similar chunks")
print("   4. Synthesize final answer")

# Test it
query = "What are the health benefits of coffee with adaptogens?"
print(f"\n🔍 Query: '{query}'")
response = advanced_qe.query(query)
print(f"\n🤖 Answer:\n{response}")

print("\n💡 Key Takeaway: Post-processing = production-quality RAG!")

# TASK 1 : Try  the same question without reranker and deduplication!


DEMO 9: Post-Processing Pipeline (Rerank + Dedup)
🔧 Setting up post-processors...


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

   ✅ Reranker (cross-encoder)
   ✅ Deduplicator (similarity threshold 0.85)
   1. Retrieve top 10 candidates
   2. Rerank with cross-encoder to top 3
   3. Deduplicate similar chunks
   4. Synthesize final answer

🔍 Query: 'What are the health benefits of coffee with adaptogens?'


2025-11-30 16:38:45,836 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-30 16:38:47,910 - INFO - AFC is enabled with max remote calls: 10.
2025-11-30 16:38:50,695 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"



🤖 Answer:
The provided information does not detail the health benefits of coffee with adaptogens. It does, however, state that the content is not medical advice and advises individuals with health conditions, those who are pregnant or nursing, or those taking medications to consult a qualified professional before trying new herbs or routines.

💡 Key Takeaway: Post-processing = production-quality RAG!


In [52]:
# =============================================================================
# DEMO 10: Custom Grounded Prompts
# =============================================================================
print("\n" + "="*70)
print("DEMO 10: Custom Grounded Prompts")
print("="*70)

from llama_index.core import PromptTemplate

# Custom prompt template with strict grounding
qa_prompt_tmpl = """
You are a coffee expert assistant. Your goal is to provide accurate,
grounded answers based ONLY on the provided context.

STRICT RULES:
1. Use ONLY the context below - no outside knowledge
2. If the answer is not in the context, say exactly: "I don't have that information in my knowledge base."
3. Keep answers to 3-5 sentences
4. Add citations after each fact
5. Be conversational but precise

Context:
{context_str}

Question: {query_str}

Answer with citations:
"""

qa_prompt = PromptTemplate(qa_prompt_tmpl)

# Apply custom prompt to query engine
custom_qe = vector_idx.as_query_engine(
    text_qa_template=qa_prompt,
    similarity_top_k=4,
    response_mode="compact"
)

print("✅ Custom grounded prompt applied")

# Test with knowledge
print("\n📝 Test 1: Question with knowledge in corpus")
query1 = "What is Ashwagandha coffee?"
response1 = custom_qe.query(query1)
print(f"Query: {query1}")
print(f"Answer: {response1}")


DEMO 10: Custom Grounded Prompts
✅ Custom grounded prompt applied

📝 Test 1: Question with knowledge in corpus


2025-11-30 16:51:41,795 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:51:41,837 - INFO - AFC is enabled with max remote calls: 10.
2025-11-30 16:51:46,884 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


Query: What is Ashwagandha coffee?
Answer: Ashwagandha coffee is a modern fusion trend that combines roasted coffee with powdered ashwagandha, an herb widely used in Indian traditions (file_path: /Users/devenderswami/GenAI/GenAI-NoteBooks/coffee_pages/01_ashwagandha_coffee_adaptogenic_latte.html). The goal is to enjoy a familiar coffee experience while incorporating earthy, slightly bitter herbal notes (file_path: /Users/devenderswami/GenAI/GenAI-NoteBooks/coffee_pages/01_ashwagandha_coffee_adaptogenic_latte.html). Home cooks often prepare it like a latte, adding spices for balance (file_path: /Users/devenderswami/GenAI/GenAI-NoteBooks/coffee_pages/01_ashwagandha_coffee_adaptogenic_latte.html). This reflects a broader movement of adapting herbs to everyday beverages (file_path: /Users/devenderswami/GenAI/GenAI-NoteBooks/coffee_pages/01_ashwagandha_coffee_adaptogenic_latte.html).


In [53]:
# Test without knowledge
print("\n📝 Test 2: Question without knowledge in corpus")
query2 = "What is quantum entanglement in coffee brewing?"
response2 = custom_qe.query(query2)
print(f"Query: {query2}")
print(f"Answer: {response2}")

print("\n💡 Key Takeaway: Custom prompts prevent hallucinations with fallback strings!")


📝 Test 2: Question without knowledge in corpus


2025-11-30 16:52:44,110 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-11-30 16:52:44,142 - INFO - AFC is enabled with max remote calls: 10.
2025-11-30 16:52:47,014 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


Query: What is quantum entanglement in coffee brewing?
Answer: I don't have that information in my knowledge base.

💡 Key Takeaway: Custom prompts prevent hallucinations with fallback strings!


In [55]:
from llama_index.core.memory import ChatMemoryBuffer

In [56]:
# Create chat engine with built-in memory
memory = ChatMemoryBuffer.from_defaults(token_limit=3000)
# Max tokens to remember!



chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt="You are a helpful coffee expert. Keep answers to 3-5 sentences."
)

print("\n🚀 Running Conversation with Built-in Memory:\n")
print("-"*80)

# Turn 1
question1 = "What is Ashwagandha coffee?"
print(f"👤 USER: {question1}")
response1 = chat_engine.chat(question1)
print(f"🤖 BOT: {response1.response}\n")


🚀 Running Conversation with Built-in Memory:

--------------------------------------------------------------------------------
👤 USER: What is Ashwagandha coffee?


2025-11-30 17:11:30,582 - INFO - AFC is enabled with max remote calls: 10.
2025-11-30 17:11:33,142 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


🤖 BOT: Ashwagandha coffee is a distinctive beverage that blends traditional coffee with adaptogenic herbs, specifically ashwagandha. This unique combination is crafted to offer sustained energy throughout the day, helping to avoid the jitters often experienced with conventional coffee. It's a popular choice within Ayurvedic wellness circles, reflecting its roots in Indian holistic health practices.



In [57]:
# Turn 2 - Uses "its" (requires memory)
question2 = "What are its health benefits?"
print(f"👤 USER: {question2}")
response2 = chat_engine.chat(question2)
print(f"🤖 BOT: {response2.response}\n")


👤 USER: What are its health benefits?


2025-11-30 17:11:53,597 - INFO - AFC is enabled with max remote calls: 10.
2025-11-30 17:11:56,647 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


🤖 BOT: Ashwagandha coffee is primarily known for its adaptogenic properties, which help the body manage stress and maintain balance. A key health benefit is providing sustained energy throughout the day, helping to prevent the common jitters or crash often associated with regular coffee consumption. This blend aims to offer a smoother, more balanced energy boost. Its popularity in Ayurvedic wellness circles further highlights its perceived benefits for overall well-being.



In [58]:
question3 = "How should I brew it?"
print(f"👤 USER: {question3}")
response3 = chat_engine.chat(question3)
print(f"🤖 BOT: {response3.response}\n")

👤 USER: How should I brew it?


2025-11-30 17:12:03,508 - INFO - AFC is enabled with max remote calls: 10.
2025-11-30 17:12:06,795 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


🤖 BOT: While specific brewing instructions for Ashwagandha coffee blends can vary, you should generally follow standard coffee brewing practices, paying close attention to water temperature. If the coffee component in your blend is a lighter roast, aim for a water temperature between 92-96°C. For darker roasts, you can use hotter water, ranging from 96-100°C. Remember that hotter water extracts more, but also increases the risk of bitterness, so adjust to your taste.

